In [1]:
import pandas as pd
import numpy as np
import sqlite3
import datetime
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, text

In [2]:
storms_csv = "Resources/storms_final.csv"
storms_sqlite = "storms_final.sqlite"
storms_df = pd.read_csv(storms_csv)
conn = sqlite3.connect(storms_sqlite)
storms_df.to_sql("storm_data", conn, if_exists='replace', index=False)
conn.close()

In [3]:
sqlite_file = "storms_final.sqlite"
conn = sqlite3.connect(sqlite_file)
df = pd.read_sql_query("SELECT * FROM storm_data", conn)

In [4]:
print(df.head())

       name combined_date  year  month  day  hour   lat  long     status  \
0  AL011852    1852-08-26  1852      8   26     6  30.2 -88.6  hurricane   
1  AL031853    1853-09-03  1853      9    3    12  19.7 -56.2  hurricane   
2  AL031854    1854-09-07  1854      9    7    12  28.0 -78.6  hurricane   
3  AL031854    1854-09-08  1854      9    8    18  31.6 -81.1  hurricane   
4  AL031854    1854-09-08  1854      9    8    20  31.7 -81.1  hurricane   

   category  wind  pressure  
0         3   100       961  
1         4   130       924  
2         3   110       938  
3         3   100       950  
4         3   100       950  


In [5]:
# Create the engine
engine = create_engine('sqlite:///storms_final.sqlite')


In [8]:
# INSPECT

# Create the inspector and connect it to the engine
inspector_gadget = inspect(engine)

# Collect the names of tables within the database
tables = inspector_gadget.get_table_names()

# print metadata for each table
for table in tables:
    print(table)
    print("-----------")
    
    # get columns
    columns = inspector_gadget.get_columns(table)
    for column in columns:
        print(column["name"], column["type"])

    print()

storm_data
-----------
name TEXT
combined_date TEXT
year INTEGER
month INTEGER
day INTEGER
hour INTEGER
lat REAL
long REAL
status TEXT
category INTEGER
wind INTEGER
pressure INTEGER



In [9]:
# Use the connection to execute the SQL query
with engine.connect() as connection:
    # Wrap the SQL query in text()
    query = text("SELECT name FROM sqlite_master WHERE type='table';")
    result = connection.execute(query)
    
    # Fetch and print the result
    tables = result.fetchall()
    print("Tables:", [row[0] for row in tables])

Tables: ['storm_data']


In [10]:
with engine.connect() as connection:
    query = text("SELECT * FROM storm_data LIMIT 10;")
    result = connection.execute(query)
    
    rows = result.fetchall()
    for row in rows:
        print(row)

('AL011852', '1852-08-26', 1852, 8, 26, 6, 30.2, -88.6, 'hurricane', 3, 100, 961)
('AL031853', '1853-09-03', 1853, 9, 3, 12, 19.7, -56.2, 'hurricane', 4, 130, 924)
('AL031854', '1854-09-07', 1854, 9, 7, 12, 28.0, -78.6, 'hurricane', 3, 110, 938)
('AL031854', '1854-09-08', 1854, 9, 8, 18, 31.6, -81.1, 'hurricane', 3, 100, 950)
('AL031854', '1854-09-08', 1854, 9, 8, 20, 31.7, -81.1, 'hurricane', 3, 100, 950)
('AL041855', '1855-08-27', 1855, 8, 27, 0, 18.4, -69.5, 'tropical storm', 0, 50, 997)
('AL011856', '1856-08-10', 1856, 8, 10, 18, 29.2, -91.1, 'hurricane', 4, 130, 934)
('AL051856', '1856-08-28', 1856, 8, 28, 6, 23.0, -82.9, 'hurricane', 2, 90, 969)
('AL021857', '1857-09-12', 1857, 9, 12, 18, 33.3, -78.3, 'hurricane', 2, 90, 961)
('AL031858', '1858-09-16', 1858, 9, 16, 18, 41.4, -72.0, 'hurricane', 1, 70, 979)


In [11]:
# Declare a Base using `automap_base()`
Base = automap_base()

# Use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)


C:\Users\eeliz\AppData\Local\Temp\ipykernel_26716\2880897608.py:5: SADeprecationWarning: The AutomapBase.prepare.reflect parameter is deprecated and will be removed in a future release.  Reflection is enabled when AutomapBase.prepare.autoload_with is passed.
  Base.prepare(engine, reflect=True)


In [12]:
# Print all of the classes mapped to the Base
print(list(Base.classes))

[]
